In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms

In [6]:
#create CNN
class CNN(nn.Module):
    
    def __init__(self,in_channels=1,num_classes=10):
        super(CNN,self).__init__()
        self.conv1=nn.Conv2d(in_channels=1,out_channels=8,kernel_size=(3,3),stride=(1,1),padding=(1,1))
        self.pool=nn.MaxPool2d(kernel_size=(2,2),stride=(2,2))
        self.conv2=nn.Conv2d(in_channels=8,out_channels=16,kernel_size=(3,3),stride=(1,1),padding=(1,1))
        self.fc1=nn.Linear(16*7*7,num_classes)

    def forward(self,x):
        x=F.relu(self.conv1(x))
        x=self.pool(x)
        x=F.relu(self.conv2(x))
        x=self.pool(x)
        x=x.reshape(x.shape[0],-1)
        x=self.fc1(x)
        return x

#set device
device='mps' if torch.backends.mps.is_available() else 'cpu'
# device='cpu'
#hyperparameter

input_size=784
num_classes=10
learning_rate=0.001
batch_size=64
num_epoch=2


#init network
model=CNN().to(device)

#loss and optimizer
loss_fn=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=learning_rate)


#train network
for epoch in range(num_epoch):
    for batch_idx, (data,targets) in enumerate(train_loader):
        #get data to gpu if avilable
        data=data.to(device)
        targets=targets.to(device)
                
        #forward
        scores=model(data)
        
        loss=loss_fn(scores,targets)
        
        #backword
        optimizer.zero_grad()
        loss.backward()
        
        #GD or adam step
        optimizer.step()

#check accuracy on both train and test set
def check_accuracy(loader,model):
    
    if loader.dataset.train:
        print("checking accuracy on train dataset")
    else:
        print("checking accuravy on test dataset")
    
    num_correct=0
    num_samples=0
    model.eval()
    
    with torch.no_grad():
        for x,y in loader:
            x=x.to(device)
            y=y.to(device)
            
            scores=model(x)
            _,pred=scores.max(1)
            num_correct+=(pred==y).sum()
            num_samples+=pred.size(0)
        print("Accuracy is :",round(float(num_correct)/float(num_samples)*100,2))
            
    model.train()
    return 

check_accuracy(train_loader,model)   
check_accuracy(test_loader,model) 

checking accuracy on train dataset
Accuracy is : 97.7
checking accuravy on test dataset
Accuracy is : 97.86
